In [88]:
import json
from subprocess import Popen
import shlex
import os
import jsonlines
from glob import glob

In [67]:
input_path = "/home/lhk/data/hp_test/hp_test_results.json"

In [106]:
with open(input_path) as file:
    data = json.load(file)

In [107]:
parent_dir, _ = os.path.split(input_path)
parent_dir += "/"

total = 0

processes = []
for entry in data:
    in_path = parent_dir+entry["result_filepath"]
    result_path, _ = os.path.split(in_path)
    
    parsed = {in_path:[]}
    for obj in jsonlines.open(in_path):
        total+= len(obj["model_completions"])
        parsed[in_path].append({"id":obj['id'], "model_completions": obj["model_completions"]})
        
    prepared_path = result_path + "/cc_eval_format.json"

    with open(prepared_path, "w") as file:
        json.dump(parsed, file)
    
    out_path = result_path + "/output.json"
    
    cmd = f"bazel run -c opt execution:solve_example -- --data_path=/home/lhk/data/ --input_path={prepared_path} --output_path={out_path}   --python3_path=/usr/bin/python3.10 --python3_library_paths=/usr/lib/python3.10"
    
    processes.append(Popen(shlex.split(cmd)))

In [108]:
total

72

In [87]:
for p in processes:
    p.wait()

I20221119 22:08:26.430264 62275 monitor.cc:386] Stack traces have been disabled
I20221119 22:08:26.902281 62458 monitor.cc:386] Stack traces have been disabled


failed
found a generation


I20221119 22:08:29.438254 62469 monitor.cc:386] Stack traces have been disabled
I20221119 22:08:29.590183 62480 monitor.cc:386] Stack traces have been disabled


failed
found a generation
compilation error
compilation error
failed
found a generation
failed
found a generation
failed
found a generation


I20221119 22:08:31.442200 62492 monitor.cc:386] Stack traces have been disabled


failed
found a generation
failed
found a generation


I20221119 22:08:31.954191 62514 monitor.cc:386] Stack traces have been disabled


passed
found a generation
failed
found a generation
compilation error
compilation error
failed
found a generation
failed
found a generation
passed
found a generation
compilation error
compilation error
failed
compilation error
found a generation
compilation error
failed
found a generation
compilation error
compilation error
failed
found a generation
failed
found a generation
compilation error
compilation error
failed
found a generation
compilation error
compilation error
failed
found a generation
compilation error
compilation error
failed
found a generation
failed
found a generation
compilation error
compilation error
failed
passed
found a generation
compilation error
compilation error
failed
found a generation
failed
found a generation
failed
found a generation
compilation error
compilation error
failed
found a generation
failed
found a generation
failed
found a generation
failed
found a generation
passed
compilation error
found a generation
compilation error
failed
compilation error


I20221119 22:08:36.966250 65390 monitor.cc:386] Stack traces have been disabled


failed
found a generation
failed
found a generation
passed
found a generation
compilation error
compilation error
failed
found a generation
compilation error
compilation error
failed
found a generation
compilation error
compilation error
failed
found a generation
compilation error
compilation error
failed
found a generation
failed
found a generation
failed
found a generation
passed
found a generation
compilation error
compilation error
failed
writing output to: /home/lhk/data/hp_test/0/frequency_penalty-0-temperature-0/output.json
writing output to: /home/lhk/data/hp_test/0/frequency_penalty-0.2-temperature-0/output.json
writing output to: /home/lhk/data/hp_test/0/frequency_penalty-0-temperature-0.1/output.json
writing output to: /home/lhk/data/hp_test/0/frequency_penalty-0.2-temperature-0.1/output.json


In [101]:
passed = 0
failed = 0

for filename in glob("/home/lhk/data/hp_test/*/*/output.json"):
    with open(filename) as file:
        data = json.load(file)
        
    results = [res['passed'] for res in data['results']]
    passed += sum(results)
    failed += len(results) - sum(results)